In [ ]:
import json
import requests
import geohash2
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 아파트, 빌라/투룸+, 원룸, 오피스텔, 창업/사무실
building_type = ["원룸", "오피스텔"]

# 서울시 행정구
Seoul_Gu = ["강남구","강동구","강북구","강서구","관악구","광진구","구로구","금천구","노원구","도봉구","동대문구","동작구","마포구","서대문구","서초구","성동구","성북구","송파구","양천구","영등포구","용산구","은평구","종로구","중구","중랑구"]

# 건물종류별/행정구별 크롤링
for building in building_type:
    for gu in Seoul_Gu:
        url = f"https://apis.zigbang.com/v2/search?leaseYn=N&q={gu}&serviceType={building}"
        res = requests.get(url)
        res.raise_for_status()
        _json = res.json()
        if _json.get("success") == True:
            items = _json.get("items")[0]
            lat = items.get("lat")
            lng = items.get("lng")
            geocode = geohash2.encode(lat, lng, precision=5)

            url = "https://apis.zigbang.com/v2/items?"\
                "deposit_gteq=0"\
                "&domain=zigbang"\
                "&geohash={}"\
                "&needHasNoFiltered=true"\
                "&rent_gteq=0"\
                "&sales_type_in=전세%7C월세"\
                "&service_type_eq={}".format(geocode, building)

            res = requests.get(url)
            res.raise_for_status()
            _json = res.json()

            _items = _json.get("items")
            item_ids = []
            for item in _items:
                item_ids.append(item.get("item_id"))

            # 매물의 개수
            print(f"{gu} {building} 매물의 개수:", len(item_ids))

            #데이터프레임으로 저장
            df = pd.DataFrame()

            for ids in item_ids:
                url=f"https://apis.zigbang.com/v2/items/{ids}"
                res=requests.get(url)
                res.raise_for_status()
                data=json.loads(res.text)

                # 크롤링 시작구간
                item = data['item']
                # 세부정보
                address = item['address']  # 상단 주소
                sales_title = item['sales_title']  # 월세
                sales_type = item['sales_type']  # 월세
                deposit = item['보증금액']
                rent = item['월세금액']
                status_check = item['상태확인At']
                area_m2 = item['전용면적_m2']  # 면적(전용)
                manage_cost = item['manage_cost']  # 관리비
                # 구조
                status = item['status']  # 아니면 진짜 오픈 상태?
                service_type = item['service_type']
                bathroom_count = item['bathroom_count']

                # 매물정보
                parking = item['parking']  # 주차
                elevator = item['elevator']  # 엘리베이터
                movein_date = item['movein_date']  # 입주가능일
                # 중복된 정보 중략
                room_direction = item['room_direction']  # 방향
                approve_date = item['approve_date']  # 준공날짜
                floor = item['floor']  # 층
                floor_all = item['floor_all']  # 건물층수
                residence_type = item['residence_type']  # 건물종류
                jibunAddress = item['jibunAddress']  # 주소

                SI = item['local1']  # 시
                GU = item['local2']  # 구
                DONG = item['local3']  # 동
                random_location = item['random_location']  # 위도, 경도

                # 리스트 생성
                s = pd.Series(
                    [ids, address, sales_title, sales_type, deposit, rent, status_check, area_m2, manage_cost, status,
                     service_type, bathroom_count,
                     parking, elevator, movein_date, room_direction, approve_date, floor, floor_all, residence_type,
                     jibunAddress, SI, GU, DONG, random_location])

                # 인근 전철역
                subways = data['subways']
                for i in range(len(subways)):
                    subway_name_i = subways[i]['name']  # 역명
                    subway_route_i = subways[i]['description']  # 호선
                    _s = pd.Series([subway_name_i, subway_route_i])
                    s = pd.concat([s, _s])
                for i in range(len(subways),3):
                    subway_name_i = None  # NaN
                    subway_route_i = None  # NaN
                    _s = pd.Series([subway_name_i, subway_route_i])
                    s = pd.concat([s, _s])

                df = df.append(s, ignore_index=True)

            df.columns = ['등록번호', '상단 주소', '전월세', '전월세', '보증금', '월세', '최신확인날짜', '면적(전용)', '관리비', '구조_오픈형', '구조_원룸','구조_욕실 개수',
                        '주차', '엘리베이터', '입주가능일', '방향', '준공날짜', '층', '건물층수', '건물종류', '지번주소', '시', '구', '동', '지도 위치',
                        '인근 지하철_역명_1', '인근 지하철_호선_1', '인근 지하철_역명_2', '인근 지하철_호선_2', '인근 지하철_역명_3', '인근 지하철_호선_3']

            df.to_excel(f'{gu} {building} 전월세 매물_직방 모바일 페이지 정보.xlsx')

            print(f"{gu} {building} 매물 크롤링 완료")